In [1]:
import torch
import numpy as np
import pandas as pd
import datetime
from libs.sequences import SeismicSequence
from libs.iris import irisRequests
from importlib import reload  # Python 3.4+
import matplotlib.pyplot as plt

In [2]:
# specify regions limits
regions = {}
regions['greece'] = (30, 45,18, 44)
regions['california'] = (30, 41, -125, -113)
regions['japan'] = (20, 50, 120, 150)
regions['italy'] = (35,46,6, 19)

In [29]:
load_cat = True
region_name = "japan"
region = regions[region_name]
if(load_cat):
    df = pd.read_csv("catalogs/" + region_name + ".csv", parse_dates=['Time'])
else:
    start_time = datetime.datetime(1980, 1, 1, 0, 0, 0)
    end_time =  datetime.datetime(2024, 1, 1, 0, 0, 0)
    download_url =irisRequests.url_events_box(start_time, end_time, region[0], region[1], region[2], region[3], minmag=3, magtype="MW")
    df = pd.read_csv(download_url, sep="|", comment="#")
    df.Time = pd.to_datetime(df.Time, errors='coerce')
    df.dropna(axis=0, inplace=True)
    df.sort_values(by="Time", inplace=True)
    df.reset_index(inplace=True, drop=True)
    df.to_csv("catalogs/japan.csv", index=False)

/var/folders/r7/3208sx_91txg405qfpwxcm3m0000gr/T/ipykernel_22086/3562696496.py:5: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("catalogs/" + region_name + ".csv", parse_dates=['Time'])


In [30]:
full_catalog = SeismicSequence.from_pandas_df(df, unit='s')

In [31]:
seqs = []
use_random_length = True
duration_days = 7
duration = 60*60*24*duration_days # in seconds
t_start = 0
max_t_end = full_catalog.t_end
while(True):
    if(use_random_length):
        t_end = t_start + np.random.exponential()*duration
    else:
        t_end = t_start + duration
    if(t_end > max_t_end):
        break
    sub_seq = full_catalog.get_subsequence(t_start, t_end)
    seqs.append(sub_seq)
    t_start = t_end
    print(len(sub_seq.arrival_times))

17
4
2
4
3
4
27
40
36
73
26
28
0
35
12
15
20
11
18
10
1
1
12
20
4
2
22
42
30
60
11
14
10
8
3
10
11
8
4
16
25
1
2
8
17
18
7
8
45
3
11
10
22
0
3
0
3
15
14
2
24
26
28
29
9
2
27
18
2
1
11
30
3
0
19
0
77
21
2
14
31
2
0
0
8
46
0
7
12
17
33
1
0
41
6
5
27
21
11
14
9
28
27
19
19
5
9
16
12
13
0
14
5
6
8
1
81
49
28
4
1
42
31
22
14
0
4
2
132
9
40
10
0
16
3
46
47
1
7
6
13
35
0
118
14
38
5
0
6
6
38
15
21
9
7
12
15
7
23
0
9
1
3
4
15
13
9
1
23
34
12
87
56
13
37
10
4
11
29
43
40
20
222
12
11
15
109
111
4
8
13
20
14
13
4
28
14
23
20
0
4
54
21
19
26
11
6
13
12
1
5
4
25
3
12
1
2
2
17
17
49
55
13
61
0
14
28
19
12
7
12
48
13
9
19
37
0
12
2
52
2
51
62
3
2
40
22
19
34
84
1
7
78
2
3
38
2
2
8
38
1
1
18
3
22
4
1
8
10
11
15
24
16
2
47
29
23
12
2
7
46
7
6
3
0
25
32
54
37
18
11
10
49
5
23
37
9
34
47
13
21
3
20
4
134
5
58
0
6
11
14
22
102
5
0
16
33
0
48
26
43
344
10
15
11
34
4
44
10
3
3
45
25
41
12
6
42
12
66
6
11
5
8
10
2
2
11
75
1
32
104
78
4
14
9
30
23
0
2
14
8
33
25
88
16
12
28
22
15
34
30
0
3
43
0
11
12
34
0
71